# FastAPI
Source: https://fastapi.tiangolo.com/learn/

## 1. First Steps

Links:
- http://127.0.0.1:8000/docs (You will see the automatic interactive API documentation - provided by Swagger UI)
- http://127.0.0.1:8000/redoc (Alternative automatic documentation - provided by ReDoc)
- http://127.0.0.1:8000/openapi.json (FastAPI automatically generates a JSON (schema) with the descriptions of all your API. You could also use it to generate code automatically, for clients that communicate with your API. For example, frontend, mobile or IoT applications.)
- https://fastapicloud.com/ (You can optionally deploy your FastAPI app with one command.) 

The simplest FastAPI file could look like this:

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def root():
    return {"message": "Hello World"}

This is our "path operation function":

- path: is /.
- operation: is get.
- function: is the function below the "decorator" (below @app.get("/")).


You can also use the other operations:

- @app.post()
- @app.put()
- @app.delete()

And the more exotic ones:

- @app.options()
- @app.head()
- @app.patch()
- @app.trace()

Steps:
- Import FastAPI.
- Create an app instance.
- Write a path operation decorator using decorators like @app.get("/").
- Define a path operation function; for example, def root(): ....
- Run the development server using the command fastapi dev.
- Optionally deploy your app with fastapi deploy.

## 2. Path Parameters

You can declare the type of a path parameter in the function, using standard Python type annotations:
It has to be an integer value

If you run this example and open your browser at http://127.0.0.1:8000/items/3, you will see a response of:

{"item_id":3}

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}

### Order matters

When creating path operations, you can find situations where you have a fixed path.

Like /users/me, let's say that it's to get data about the current user.

And then you can also have a path /users/{user_id} to get data about a specific user by some user ID.

Because path operations are evaluated in order, you need to make sure that the path for /users/me is declared before the one for /users/{user_id}:

Cause the operation used will always be the one that matches first.

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/users/me")
async def read_user_me():
    return {"user_id": "the current user"}


@app.get("/users/{user_id}")
async def read_user(user_id: str):
    return {"user_id": user_id}

### Predefined values for Parameters

#### Create an Enum class: ¶

- Import Enum and create a sub-class that inherits from str and from Enum.

- By inheriting from str the API docs will be able to know that the values must be of type string and will be able to render correctly.

- Then create class attributes with fixed values, which will be the available valid values

- Predefined values are showed on interactive docs

- class ModelName is an enumeration

In [ ]:
from enum import Enum

from fastapi import FastAPI


class ModelName(str, Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet"


app = FastAPI()


@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name is ModelName.alexnet:
        return {"model_name": model_name, "message": "Deep Learning FTW!"}

    if model_name.value == "lenet":
        return {"model_name": model_name, "message": "LeCNN all the images"}

    return {"model_name": model_name, "message": "Have some residuals"}

In your client you will get a JSON response like:

In [ ]:
{
  "model_name": "alexnet",
  "message": "Deep Learning FTW!"
}

If you need a parameter to contain a path inside that is possible, but docs won't tell that the parameter should contain a entire path.

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/files/{file_path:path}")
async def read_file(file_path: str):
    return {"file_path": file_path}

## 3. Query Parameters

When you declare other function parameters that are not part of the path parameters, they are automatically interpreted as "query" parameters.

The query is the set of key-value pairs that go after the ? in a URL, separated by & characters.

For example, in the URL:


- http://127.0.0.1:8000/items/?skip=0&limit=10
...the query parameters are:

- skip: with a value of 0
- limit: with a value of 10

  
As they are part of the URL, they are "naturally" strings. But when you declare them with Python types (in the example in the next cell, as int), they are converted to that type and validated against it.

In [ ]:
from fastapi import FastAPI

app = FastAPI()

fake_items_db = [{"item_name": "Foo"}, {"item_name": "Bar"}, {"item_name": "Baz"}]


@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return fake_items_db[skip : skip + limit]

### Defaults

In the example above they have default values of skip=0 and limit=10. So these two are the same:
- http://127.0.0.1:8000/items/
- http://127.0.0.1:8000/items/?skip=0&limit=10

But this one will use default limit set on code (10):
- http://127.0.0.1:8000/items/?skip=20



You can declare optional query parameters, by setting their default to None

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def read_item(item_id: str, q: str | None = None, short: bool = False): # You can also declare bool types, and they will be converted:
    # In this case, the function parameter q will be optional, and will be None by default.
    # Also notice that FastAPI is smart enough to notice that the path parameter item_id is a path parameter and q is not, so, it's a query parameter.
    item = {"item_id": item_id}
    if q:
        item.update({"q": q})
    if not short:
        item.update(
            {"description": "This is an amazing item that has a long description"}
        )
    return item

Just to make it clear:
- you can define some parameters as required, some as having a default value, and some entirely optional

In this case, there are 3 query parameters:

- needy, a required str.
- skip, an int with a default value of 0.
- limit, an optional int.

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def read_user_item(
    item_id: str, needy: str, skip: int = 0, limit: int | None = None
):
    item = {"item_id": item_id, "needy": needy, "skip": skip, "limit": limit}
    return item

## 4. Pydantic for Request Body

- A request body is data sent by the client to your API. (don't necessarily need to send request bodies all the time)
- A response body is the data your API sends to the client. (almost always)

- To declare a request body, you use Pydantic models with all their power and benefits. Here are the benefits:

With just that Python type declaration, FastAPI will

- Read the body of the request as JSON.
- Convert the corresponding types (if needed).
- Validate the data: If the data is invalid, it will return a nice and clear error, indicating exactly where and what was the incorrect data.
- Give you the received data in the parameter item: As you declared it in the function to be of type Item, you will also have all the editor support (completion, etc) for all of the attributes and their types.
- Generate JSON Schema definitions for your model, you can also use them anywhere else you like if it makes sense for your project.
- Those schemas will be part of the generated OpenAPI schema, and used by the automatic documentation UIs.


Tip: If you use PyCharm as your editor, you can use the Pydantic PyCharm Plugin. It improves editor support for Pydantic models, with:

- auto-completion
- type checks
- refactoring
- searching
- inspections

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


app = FastAPI()


@app.post("/items/")
async def create_item(item: Item):
    return item

Inside of the function, you can access all the attributes of the model object directly:

In [ ]:
(...)

@app.post("/items/")
async def create_item(item: Item):
    item_dict = item.model_dump()
    if item.tax is not None:
        price_with_tax = item.price + item.tax
        item_dict.update({"price_with_tax": price_with_tax})
    return item_dict

### Request body + path parameters

You can declare path parameters and request body at the same time.

FastAPI will recognize that the function parameters that match path parameters should be taken from the path, and that function parameters that are declared to be Pydantic models should be taken from the request body.

In [ ]:
@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Item):
    return {"item_id": item_id, **item.model_dump()}

### Request body + path + query parameters

In [ ]:
@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Item, q: str | None = None):
    result = {"item_id": item_id, **item.model_dump()}
    if q:
        result.update({"q": q})
    return result

## 5. Query Parameters and String Validations